In [2]:
import numpy as np  
import sys,os  
caffe_root = '/data/ssd-caffe/yolov3-caffe/'
sys.path.insert(0, caffe_root + 'python')  
import caffe  

#train_proto = 'yolov3-tiny-ncs-without-last-maxpool.prototxt'
#train_model = 'snapshot/mobilenet_iter_7000.caffemodel'  #should be your snapshot caffemodel
train_proto = 'yolov3.prototxt'
deploy_proto = 'yolov3_merged_bn.prototxt'
'''if(len(sys.argv) < 2):
    print("please input caffemodel file from commandLine")
    exit(1)
train_model = sys.argv[1]'''

#deploy_proto = 'yolov3-tiny-ncs-without-last-maxpool-merg-batchnorm.prototxt'  
#save_model = 'iter_7000_MobileNetSSD_deploy.caffemodel'
#save_model = train_model.split('/')[-1] + "MobileNetSSD_deploy.caffemodel"
save_model = "generated_merge_yolov3.caffemodel"
#global buf #----why can not work ?

def merge_bn(net, nob):
    '''
    merge the batchnorm, scale layer weights to the conv layer, to  improve the performance
    var = var + scaleFacotr
    rstd = 1. / sqrt(var + eps)
    w = w * rstd * scale
    b = (b - mean) * rstd * scale + shift
    '''
    buf = [0, 2, 0, 960000, 0]
    for key in net.params.iterkeys():
        print("key:" + str(key))
        if type(net.params[key]) is caffe._caffe.BlobVec:
            print("is caffe._caffe.BlobVec")
            if key.endswith("-bn") or key.endswith("-scale"):
                continue
            else:
                conv = net.params[key]
                print("merge layer {0}".format(conv))
                if not net.params.has_key(key[:-5] + "-bn"):
                    print("not need bn, copy w and b")
                    for i, w in enumerate(conv):
                        #print("i={0}".format(i))
                        nob.params[key][i].data[...] = w.data
                        #print(w.data.shape)
                    temp_array = (conv[1].data).reshape(-1)                       
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                    temp_array = (conv[0].data).reshape(-1)                       
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                else:
                    
                    print("need bn, really start bn and scale")
                    bn = net.params[key[:-5] + "-bn"]
                    scale = net.params[key[:-5] + "-scale"]
                    wt = conv[0].data
                    channels = wt.shape[0]
                    bias = np.zeros(wt.shape[0])
                    if len(conv) > 1:
                        bias = conv[1].data
                    mean = bn[0].data
                    var = bn[1].data
                    scalef = bn[2].data
                    print('scalef = {0}'.format(scalef))

                    scales = scale[0].data
                    shift = scale[1].data

                    if scalef != 0:
                        scalef = 1. / scalef
                    mean = mean * scalef
                    var = var * scalef
                    rstd = 1. / np.sqrt(var + 1e-5)
                    rstd1 = rstd.reshape((channels,1,1,1))
                    scales1 = scales.reshape((channels,1,1,1))
                    wt = wt * rstd1 * scales1
                    bias = (bias - mean) * rstd * scales + shift                   
                    nob.params[key][0].data[...] = wt                   
                    nob.params[key][1].data[...] = bias
                    
                    print(bias.shape)
                    temp_array = bias.reshape(-1)
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                    
                    print(wt.shape)
                    temp_array = wt.reshape(-1)
                    buf = buf + temp_array.tolist()
                    print(len(buf))
    weights_merged_bn = np.array(buf, dtype=np.float32)
    print(weights_merged_bn)
    fp = open('aaa_yolov3.weights', "wb")
    weights_merged_bn.tofile(fp)
    #np.save('npsaved_file', weights_merged_bn)

#net = caffe.Net(train_proto, train_model, caffe.TRAIN)
caffe.set_mode_cpu()
#net = caffe.Net(train_proto, 'Jenerated_nolastpooling.caffemodel', caffe.TEST)
net = caffe.Net(train_proto, 'yolov3.caffemodel', caffe.TEST)
net_deploy = caffe.Net(deploy_proto, caffe.TEST)  

merge_bn(net, net_deploy)
#net_deploy.save(save_model)


key:layer1-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa161a7bec0>
need bn, really start bn and scale
scalef = [ 1.]
(32,)
37
(32, 3, 3, 3)
901
key:layer1-bn
is caffe._caffe.BlobVec
key:layer1-scale
is caffe._caffe.BlobVec
key:layer2-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa161a7bd70>
need bn, really start bn and scale
scalef = [ 1.]
(64,)
965
(64, 32, 3, 3)
19397
key:layer2-bn
is caffe._caffe.BlobVec
key:layer2-scale
is caffe._caffe.BlobVec
key:layer3-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa161a7b050>
need bn, really start bn and scale
scalef = [ 1.]
(32,)
19429
(32, 64, 1, 1)
21477
key:layer3-bn
is caffe._caffe.BlobVec
key:layer3-scale
is caffe._caffe.BlobVec
key:layer4-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa161a7b1a0>
need bn, really start bn and scale
scalef = [ 1.]
(64,)
21541
(64, 32, 3, 3)
39973
key:layer4-bn
is caffe._caffe.BlobVec
k

key:layer45-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa15311c2f0>
need bn, really start bn and scale
scalef = [ 1.]
(256,)
7001125
(256, 512, 1, 1)
7132197
key:layer45-bn
is caffe._caffe.BlobVec
key:layer45-scale
is caffe._caffe.BlobVec
key:layer46-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa15311c440>
need bn, really start bn and scale
scalef = [ 1.]
(512,)
7132709
(512, 256, 3, 3)
8312357
key:layer46-bn
is caffe._caffe.BlobVec
key:layer46-scale
is caffe._caffe.BlobVec
key:layer48-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa15311c590>
need bn, really start bn and scale
scalef = [ 1.]
(256,)
8312613
(256, 512, 1, 1)
8443685
key:layer48-bn
is caffe._caffe.BlobVec
key:layer48-scale
is caffe._caffe.BlobVec
key:layer49-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa15311c6e0>
need bn, really start bn and scale
scalef = [ 1.]
(512,)
8444197
(512, 256, 3, 3)
9

key:layer89-bn
is caffe._caffe.BlobVec
key:layer89-scale
is caffe._caffe.BlobVec
key:layer90-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa153121b40>
need bn, really start bn and scale
scalef = [ 1.]
(256,)
58070308
(256, 512, 1, 1)
58201380
key:layer90-bn
is caffe._caffe.BlobVec
key:layer90-scale
is caffe._caffe.BlobVec
key:layer91-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa153121c90>
need bn, really start bn and scale
scalef = [ 1.]
(512,)
58201892
(512, 256, 3, 3)
59381540
key:layer91-bn
is caffe._caffe.BlobVec
key:layer91-scale
is caffe._caffe.BlobVec
key:layer92-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa153121de0>
need bn, really start bn and scale
scalef = [ 1.]
(256,)
59381796
(256, 512, 1, 1)
59512868
key:layer92-bn
is caffe._caffe.BlobVec
key:layer92-scale
is caffe._caffe.BlobVec
key:layer93-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7fa153121f

In [ ]:
help(np.ndarray.tofile)